# OWL API tutorial: loading and editing OWL ontologies 

This is a Jupyter notebook tutorial to demonstrate how to use the OWL API to load and manipulate OWL ontologies.

**Author:** Kody Moodley

**License:** [Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)](https://creativecommons.org/licenses/by-nc-sa/4.0/)

**Requirements:**

* Jupyter notebooks: **https://jupyter.org/**
* IJava: **https://github.com/SpencerPark/IJava** (Java kernel for Jupyter notebooks)

For more details about how to use the OWL API, consult the documentation on Github: **https://github.com/owlcs/owlapi/wiki/Documentation**

The API (Java) docs for OWL API version 4 are located here: **http://owlcs.github.io/owlapi/apidocs_4/** and those for OWL API version 5 are located here: **http://owlcs.github.io/owlapi/apidocs_5/**

## Part A. Setup environment

#### 1. Import OWL API
First we need to import the OWL API Java library. We can do this using the command below which uses Jupyter cell magic to import the library by specifying its Maven POM description. We obtain this description by searching for the OWL API at: **https://mvnrepository.com/**

In [1]:
%%loadFromPOM
<dependency>
    <groupId>net.sourceforge.owlapi</groupId>
    <artifactId>owlapi-distribution</artifactId>
    <version>5.1.16</version>
</dependency>

#### 2. Import major classes from OWL API
Next, we need to import four central and important classes from the OWL API which enable us to perform the major loading and editing tasks.

* An ``OWLOntology`` instance obviously represents an OWL ontology that you can save and manipulate using the OWL API.
* ``OWLManager`` provides access to an ``OWLOntologyManager`` and ``OWLDataFactory``.
* ``OWLOntologyManager`` enables you to load, store and save OWL ontologies to file. It also allows to add / remove axioms (logical statements) from the ontologies. You can load multiple ontologies into one ``OWLOntologyManager`` instance.
* ``OWLDataFactory`` provides access to methods that allow you to create new Classes (``OWLClass`` and ``OWLClassExpression`` objects), relations (``OWLObjectProperty`` objects), instances (``OWLIndividual`` objects), axioms (``OWLAxiom`` objects) etc. 


In [2]:
import org.semanticweb.owlapi.apibinding.OWLManager;
import org.semanticweb.owlapi.model.OWLOntology;
import org.semanticweb.owlapi.model.OWLDataFactory;
import org.semanticweb.owlapi.model.OWLOntologyManager;

#### 3. Create an instance of OWLOntologyManager

In [3]:
OWLOntologyManager manager = OWLManager.createOWLOntologyManager();

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


## Part B. Loading ontologies

#### 1a. Load OWL ontology from file

How to load an OWL ontology from a .owl file stored on your local computer 

In [4]:
// Obtain a reference to a local OWL ontology file
File file = new File("ontologies/pizza.owl");
// Load the ontology from file into an ontology manager
OWLOntology local_ontology = manager.loadOntologyFromOntologyDocument(file);

#### 1b. Load OWL ontology from remote URI (where the ontology is hosted)

How to load an ontology that is hosted remotely on the Web

In [5]:
//Import IRI (representing full identifier for an ontology or construct in the ontology)
import org.semanticweb.owlapi.model.IRI;
//Specify the URI where the ontology is located
IRI remoteOntology = IRI.create("https://raw.githubusercontent.com/micheldumontier/semanticscience/master/ontology/sio/release/sio-release.owl");
//Load the ontology from the URI into the ontology manager
OWLOntology local_ontology = manager.loadOntology(remoteOntology);

#### 2. Print some metadata about the ontology

Metadata about an ontology is information about its creation and maintenance. For example: who created it? when was it created? what is the purpose behind it? what are the topics it addresses? etc. This metadata is stored as OWL annotations (also called OWL annotation axioms) in the ontology file itself. The code in the cell below retrieves these statements and prints them.

In [6]:
// Import OWLAnnotation interface
import org.semanticweb.owlapi.model.OWLAnnotation;
import org.semanticweb.owlapi.model.OWLAnnotationValue;
// Import OWL Literal (annotation data value type)
import org.semanticweb.owlapi.model.OWLLiteral;

String ont_name = "";
String ont_desc = "";
String ont_authors = "| ";
String ont_version = "";
String ont_provenance = "";
String ont_license = "";

// Loop through all annotations in ontology and check for which annotation property type it is. Based on that type, we can determine what kind of metadata that is and print the value for the properties
for (OWLAnnotation annotation : local_ontology.getAnnotations()){
    if (annotation.getProperty().toStringID().equals("http://purl.org/dc/elements/1.1/description") || annotation.getProperty().toStringID().equals("http://purl.org/dc/terms/description")){
        if (annotation.getValue() instanceof OWLLiteral) {
            OWLLiteral val = (OWLLiteral) annotation.getValue();
            ont_desc = val.getLiteral();
        }
        else {
            ont_desc = annotation.getValue().toString();
        }
    }
    if (annotation.getProperty().toStringID().equals("http://purl.org/dc/elements/1.1/title") || annotation.getProperty().toStringID().equals("http://purl.org/dc/terms/title")){
        if (annotation.getValue() instanceof OWLLiteral) {
            OWLLiteral val = (OWLLiteral) annotation.getValue();
            ont_name = val.getLiteral();
        }
        else {
            ont_name = annotation.getValue().toString();
        }
    }
    if (annotation.getProperty().toStringID().equals("http://purl.org/dc/terms/provenance")){
        if (annotation.getValue() instanceof OWLLiteral) {
            OWLLiteral val = (OWLLiteral) annotation.getValue();
            ont_provenance = val.getLiteral();
        }
        else {
            ont_provenance = annotation.getValue().toString();
        }
    }
    if (annotation.getProperty().toStringID().equals("http://purl.org/dc/terms/license")){
        if (annotation.getValue() instanceof OWLLiteral) {
            OWLLiteral val = (OWLLiteral) annotation.getValue();
            ont_license = val.getLiteral();
        }
        else {
            ont_license = annotation.getValue().toString();
        }
    }
    if (annotation.getProperty().toStringID().equals("http://purl.org/dc/terms/contributor") || 
        annotation.getProperty().toStringID().equals("http://purl.org/dc/elements/1.1/contributor") || 
        annotation.getProperty().toStringID().equals("http://purl.org/dc/elements/1.1/creator") ||
        annotation.getProperty().toStringID().equals("http://purl.org/dc/terms/creator")){
        if (annotation.getValue() instanceof OWLLiteral) {
            OWLLiteral val = (OWLLiteral) annotation.getValue();
            ont_authors += val.getLiteral() + " | ";
        }
        else {
            ont_authors += annotation.getValue().toString() + " | ";
        }
    }
    if (annotation.getProperty().toStringID().equals("http://www.w3.org/2002/07/owl#versionInfo")){
        if (annotation.getValue() instanceof OWLLiteral) {
            OWLLiteral val = (OWLLiteral) annotation.getValue();
            ont_version = val.getLiteral();
        }
        else {
            ont_version = annotation.getValue().toString();
        }
    }
}

System.out.println("Title: " + ont_name);
System.out.println();
System.out.println("Description: " + ont_desc);
System.out.println();
System.out.println("Authors: " + ont_authors);
System.out.println();
System.out.println("License: " + ont_license);
System.out.println();
System.out.println("Version: " + ont_version);
System.out.println();
System.out.println("Provenance: " + ont_provenance);
System.out.println();

Title: Semanticscience Integrated Ontology (SIO)

Description: The semanticscience integrated ontology (SIO) provides a simple, integrated ontology (types, relations) for objects, processes and their attributes.

This project provides foundational support for the Bio2RDF (http://bio2rdf.org) and SADI (http://sadiframework.org) projects. 

website: http://semanticscience.org
email: sio-ontology@googlegroups.com
mailing list: http://groups.google.com/group/sio-ontology


Authors: | Contributors are those that engage in discussions in the context of SIO (in alphabetical order):
sivaram arabandi
christopher baker
joachim baran
jerven bolleman
alison callahan
leonid chepelev
kevin cohen
melanie courtot
geraint duck
laura furlong
elisa kendall
tatsuya kushida
luke mccarthy
jim mccusker
jose miguel cruz-toledo
robert hoehndorf
simon jupp
jin-dong kim
dana klassen
thomas luetteke
james malone
chris mungall
david osumi-sutherland
tazro ohta
nuria queralt
stephen reed
alexandre riazanov
matthias

#### 3. Print some statistics about the ontology

Print some statistics about the ontology, like how many logical statements, classes, relations etc. it contains. 

In [7]:
// Number of axioms and constructs in ontology
System.out.println("Number of axioms: " + local_ontology.getAxiomCount());
System.out.println("Number of logical axioms: " + local_ontology.getLogicalAxiomCount());
System.out.println("Number of classes: " + local_ontology.getClassesInSignature(true).size());
System.out.println("Number of relations: " + local_ontology.getObjectPropertiesInSignature(true).size());
System.out.println("Number of instances: " + local_ontology.getIndividualsInSignature(true).size());

// Import AxiomType class which enumerates different types of OWLAxiom
import org.semanticweb.owlapi.model.AxiomType;

// Number of axioms of a specific type in ontology
System.out.println("Number of SubClassOf axioms: " + local_ontology.getAxioms(AxiomType.SUBCLASS_OF).size());
System.out.println("Number of EquivalentClasses axioms: " + local_ontology.getAxioms(AxiomType.EQUIVALENT_CLASSES).size());
System.out.println("Number of DisjointClasses axioms: " + local_ontology.getAxioms(AxiomType.DISJOINT_CLASSES).size());
System.out.println("Number of Class assertions: " + local_ontology.getAxioms(AxiomType.CLASS_ASSERTION).size());
System.out.println("Number of ObjectProperty assertions: " + local_ontology.getAxioms(AxiomType.OBJECT_PROPERTY_ASSERTION).size());

Number of axioms: 12673
Number of logical axioms: 2502
Number of classes: 1552
Number of relations: 211
Number of instances: 0
Number of SubClassOf axioms: 1927
Number of EquivalentClasses axioms: 49
Number of DisjointClasses axioms: 81
Number of Class assertions: 0
Number of ObjectProperty assertions: 0


## Part C. Building an OWL ontology

#### 1. Create a blank (empty) ontology
First we need to create a fresh ontology with no axioms, classes or relations yet. We have to give it a unique Internationalized Resource Identifier (IRI) as well. Let the domain / topic of our ontology be about family relationships. The IRI specified should comply with the formal specification for IRI's developed by the IETF (Internet Engineering Task Force) located here: [https://www.ietf.org/rfc/rfc3987.txt](https://www.ietf.org/rfc/rfc3987.txt) The OWL API will automatically validate your IRI and will return a compile error if your IRI is invalid.

In [8]:
// Import IRI class
import org.semanticweb.owlapi.model.IRI;
// Create an IRI for our ontology. This can be any valid HTTP URI
IRI ontologyIRI = IRI.create("http://maastrichtuniversity.nl/ids/owlapi-tutorial/family-relationsips");
// Create the ontology by using the createOntology method of the ontology manager (there a number of constructors for this class but we will use the one which requires only an IRI)
OWLOntology ont = manager.createOntology(ontologyIRI);

In [9]:
System.out.println("Number of axioms: " + ont.getAxiomCount());

Number of axioms: 0


**Voila! We have created a blank OWL ontology!**
Okay, but that's not very interesting... Let us add some information to the ontology.

#### 2. Create an instance of OWLDataFactory
Remember the ``OWLDataFactory`` class is one of the central classes in the OWL API and allows us to create new content (classes, instances, relations etc.) to add to the ontology.

In [10]:
// Get OWLDataFactory instance from OWLOntologyManager instance.
OWLDataFactory factory = manager.getOWLDataFactory();

#### 3. Create some individuals to add to our ontology

Here we use the ``getOWLNamedIndividual`` method from the ``OWLDataFactory`` class to create individuals. The parameter required is an IRI of the new individual(s). When adding any entity (i.e. class, relation or individual) to the ontology, the established practice is to formulate the IRI for this new entity by appending the "#" or "/" character to the IRI of the ontology followed by the chosen name for the new entity. The name of the entity itself (also called ``resource name``) is up to ontology engineer to decide. It is a commonly used (but not required) practice to use lowercase letters for individual names.

In [11]:
// Import OWLIndividual class
import org.semanticweb.owlapi.model.OWLIndividual;
// Create some instances (individuals)
OWLIndividual kody = factory.getOWLNamedIndividual(IRI.create(ontologyIRI + "#kody"));
OWLIndividual michel = factory.getOWLNamedIndividual(IRI.create(ontologyIRI + "#michel"));
OWLIndividual tiffany = factory.getOWLNamedIndividual(IRI.create(ontologyIRI + "#tiffany"));
OWLIndividual nicole = factory.getOWLNamedIndividual(IRI.create(ontologyIRI + "#nicole"));
System.out.println(kody);
System.out.println(michel);
System.out.println(tiffany);
System.out.println(nicole);

<http://maastrichtuniversity.nl/ids/owlapi-tutorial/family-relationsips#kody>
<http://maastrichtuniversity.nl/ids/owlapi-tutorial/family-relationsips#michel>
<http://maastrichtuniversity.nl/ids/owlapi-tutorial/family-relationsips#tiffany>
<http://maastrichtuniversity.nl/ids/owlapi-tutorial/family-relationsips#nicole>


#### 4. Create a relation (Object Property) to add to our ontology

Here we use the ``getOWLObjectProperty`` method from the ``OWLDataFactory`` class to create new relations. The parameter required is again an IRI of the new relation. It is common practice (but not mandatory) to use camel case with a lowercase first letter for Object properties in OWL.

In [12]:
// Import OWLObjectProperty class 
import org.semanticweb.owlapi.model.OWLObjectProperty;
// Create a role called "hasWife"
OWLObjectProperty hasWife = factory.getOWLObjectProperty(IRI.create(ontologyIRI + "#hasWife"));
System.out.println(hasWife);

<http://maastrichtuniversity.nl/ids/owlapi-tutorial/family-relationsips#hasWife>


#### 5. Create an ObjectPropertyAssertion axiom to add to the ontology

Here we use the ``getOWLObjectPropertyAssertionAxiom`` method from the ``OWLDataFactory`` class to create new role assertions. The parameters required are 1) the relation, 2) entity1 and 3) entity2 which are related via the relation. **NB:** relation assertion axioms are directional and are not necessarily symmetric.  

In [13]:
// Import OWLObjectPropertyAssertionAxiom class 
import org.semanticweb.owlapi.model.OWLObjectPropertyAssertionAxiom;
// Create axioms
// NB: this means that "michel hasWife tiffany", the order of parameters is important! if we swapped michel and tiffany, the meaning of the axiom would be "tiffany hasWife michel"!
OWLObjectPropertyAssertionAxiom axiom1 = factory.getOWLObjectPropertyAssertionAxiom(hasWife, michel, tiffany);
// NB: this means that "kody hasWife nicole", the order of parameters is important! if we swapped kody and nicole, the meaning of the axiom would be "nicole hasWife kody"!
OWLObjectPropertyAssertionAxiom axiom2 = factory.getOWLObjectPropertyAssertionAxiom(hasWife, kody, nicole);
System.out.println(axiom1);
System.out.println(axiom2);

ObjectPropertyAssertion(<http://maastrichtuniversity.nl/ids/owlapi-tutorial/family-relationsips#hasWife> <http://maastrichtuniversity.nl/ids/owlapi-tutorial/family-relationsips#michel> <http://maastrichtuniversity.nl/ids/owlapi-tutorial/family-relationsips#tiffany>)
ObjectPropertyAssertion(<http://maastrichtuniversity.nl/ids/owlapi-tutorial/family-relationsips#hasWife> <http://maastrichtuniversity.nl/ids/owlapi-tutorial/family-relationsips#kody> <http://maastrichtuniversity.nl/ids/owlapi-tutorial/family-relationsips#nicole>)


#### 6. Adding and removing axioms

Here we use the ``addAxiom`` and ``removeAxiom`` methods from the ``OWLOntologyManager`` class to add/remove axioms to an ontology. The parameters required are 1) the ontology to the add the axiom to / remove the axiom from (remember we can have multiple ontologies loaded in a single ontology manager) and 2) the axiom to add/remove.

In [14]:
// Add axioms one at a time
manager.addAxiom(ont, axiom1);
manager.addAxiom(ont, axiom2);

System.out.println("Number of axioms: " + ont.getAxiomCount());

// Remove axioms one at a time
manager.removeAxiom(ont, axiom1);
manager.removeAxiom(ont, axiom2);

System.out.println("Number of axioms (after axioms removed): " + ont.getAxiomCount());

// Import OWLAxiom class 
import org.semanticweb.owlapi.model.OWLAxiom;
// Import Java Set
import java.util.Set;
// Import Java HashSet
import java.util.HashSet;

// Batch add axioms: you can also batch add / remove by putting all axioms to add / remove into a Java Set (HashSet).
Set<OWLAxiom> axioms_to_add = new HashSet<OWLAxiom>();
axioms_to_add.add(axiom1);
axioms_to_add.add(axiom2);
manager.addAxioms(ont, axioms_to_add);

System.out.println("Number of axioms (after batch add): " + ont.getAxiomCount());

// Batch remove axioms
manager.removeAxioms(ont, axioms_to_add);

System.out.println("Number of axioms (after batch remove): " + ont.getAxiomCount());

// Put the axioms back so we can continue with the tutorial! :)
manager.addAxioms(ont, axioms_to_add);

System.out.println("Number of axioms (final): " + ont.getAxiomCount());

Number of axioms: 2
Number of axioms (after axioms removed): 0
Number of axioms (after batch add): 2
Number of axioms (after batch remove): 0
Number of axioms (final): 2


#### 7. Data properties

Data properties in OWL allow one to express relations / properties of an instance that have values which have a concrete data type (e.g. String, Integer, Float, boolean etc.) They are distinct from Object properties as we saw earlier which are relations between instances (i.e. both entities being related are instances / objects from the domain). Examples of data properties that may be relevant for people are: their age, height and weight etc. It is common practice (but not mandatory) to use camel case with a lowercase first letter for Data properties in OWL.

In [15]:
// Import OWLDataProperty class 
import org.semanticweb.owlapi.model.OWLDataProperty;
// Create a "hasAge" data property
OWLDataProperty hasAge = factory.getOWLDataProperty(IRI.create(ontologyIRI + "#hasAge"));
System.out.println(hasAge);
// Create data property assertion to state that "kody hasAge 34"
OWLAxiom axiom4 = factory.getOWLDataPropertyAssertionAxiom(hasAge, kody, 34);
System.out.println(axiom4);
manager.addAxiom(ont, axiom4);
// In the above code, 34 is an integer, so we can just pass 34 into the
// data factory method. Behind the scenes the OWL API will create a
// typed constant that it will use as the value of the data property
// assertion.
// Notice from the output how "34" is automatically recognized by OWL API to be an integer value

<http://maastrichtuniversity.nl/ids/owlapi-tutorial/family-relationsips#hasAge>
DataPropertyAssertion(<http://maastrichtuniversity.nl/ids/owlapi-tutorial/family-relationsips#hasAge> <http://maastrichtuniversity.nl/ids/owlapi-tutorial/family-relationsips#kody> "34"^^xsd:integer)


SUCCESSFULLY

#### 8. Classes

Now we can add some categories (called classes or concepts in OWL) to our ontology. For example, kody and michel are instances of the category ``Person``.

In [16]:
// Import OWLClass class 
import org.semanticweb.owlapi.model.OWLClass;

In [17]:
// Complex classes (OWLClassExpression)

In [18]:
// Domain and range of roles

In [19]:
// Range of data properties

In [20]:
// Equivalence axioms

In [21]:
// Disjoint axioms

In [22]:
// SubClass axioms

In [23]:
// OWLAnnotations (metadata about our ontology)

In [24]:
// Add extra info

// OWLObjectProperty hasSon = factory.getOWLObjectProperty(IRI.create(ontologyIRI + "#hasSon"));
// // Create the assertion, John hasSon Bill
// OWLAxiom axiom2 = factory.getOWLObjectPropertyAssertionAxiom(hasSon, john, bill);
// // Apply the change
// manager.applyChange(new AddAxiom(ont, axiom2));
// // John hasDaughter Susan
// OWLObjectProperty hasDaughter = factory.getOWLObjectProperty(IRI.create(ontologyIRI + "#hasDaughter"));
// OWLAxiom axiom3 = factory.getOWLObjectPropertyAssertionAxiom(hasDaughter, john, susan);
// manager.applyChange(new AddAxiom(ont, axiom3));

In [25]:
// // Saving ontologies - concrete OWL Syntaxes
// manager.saveOntology(ont, new StringDocumentTarget());
// // OWL/XML
// // System.out.println("OWL/XML: ");
// manager.saveOntology(ont, new OWLXMLDocumentFormat(), new StringDocumentTarget());
// // Manchester Syntax
// // System.out.println("Manchester syntax: ");
// manager.saveOntology(ont, new ManchesterSyntaxDocumentFormat(), new StringDocumentTarget());
// // Turtle
// // System.out.println("Turtle: ");
// manager.saveOntology(ont, new TurtleDocumentFormat(), new StringDocumentTarget());

In [26]:
// Visualisation (WebVowel)

In [27]:
// Reasoning

In [28]:
// Classification

In [29]:
// TBox subsumption

In [30]:
// Unsatisfiable classes

In [31]:
// Realisation (infer class)

In [32]:
// Inconsistent ontologies

In [33]:
// %%loadFromPOM
// <dependency>
//     <groupId>com.hermit-reasoner</groupId>
//     <artifactId>org.semanticweb.hermit</artifactId>
//     <version>1.3.8.4</version>
// </dependency>

In [34]:
%%loadFromPOM
<dependency>
    <groupId>net.sourceforge.owlapi</groupId>
    <artifactId>jfact</artifactId>
    <version>5.0.3</version>
</dependency>

In [35]:
import uk.ac.manchester.cs.jfact.JFactFactory;
import org.semanticweb.owlapi.reasoner.OWLReasonerFactory;
import org.semanticweb.owlapi.reasoner.OWLReasoner;

OWLReasonerFactory rf = new JFactFactory();
OWLReasoner jfact = rf.createReasoner(local_ontology);
// import org.semanticweb.HermiT.Reasoner;
// Reasoner hermit=new Reasoner(local_ontology);
System.out.println(jfact.isConsistent());

true
